# Gathering the Tabular Data for Non-Winning songs

I recently found a website called *[KPOPDB](https://www.kpopdb.net/en/)* that contains not only the initial music show wins data from 2019 but the competition for each win. This provides more data as to how close a win was, the competitveness of a given week, but more importantly gives us an idea of songs that have never won a music show despite being in contention of one. If it appears that I require more songs, I will resort to scraping a release chart of different comebacks to consider in my analysis.

In [ ]:
import requests
from bs4 import BeautifulSoup
import csv
import re

def sanitize_filename(name: str) -> str:
    """
    Replace characters that aren't safe for filenames.
    E.g., for "Music Bank?" -> "Music_Bank_"
    """
    return re.sub(r'[^\w\-\._ ]', '_', name)

def scrape_kpopdb_year(year: int):
    """
    Scrape https://www.kpopdb.net/en/wins.php?y={year}
    and create one CSV per show (e.g., 'Music Bank') 
    containing a header row + all placements.
    """
    base_url = "https://www.kpopdb.net/en/wins.php"
    resp = requests.get(f"{base_url}?y={year}")
    soup = BeautifulSoup(resp.text, "html.parser")
    
    # Collect all <tr> in the main table
    all_rows = soup.select("table.table tbody tr")
    
    # We'll group our data by show name 
    # so we can write each show to its own CSV file
    data_by_show = {}
    
    current_show_name = ""
    current_date = ""
    
    for row in all_rows:
        row_id = row.get("id", "")
        
        # ---------------------------------------------------------------------
        # 1) If this <tr> is the "detail" row, it has a nested <table>:
        #    <tr id="detail###">
        #      <td colspan="...">
        #        <table> ... header row + data rows ... </table>
        #      </td>
        #    </tr>
        # ---------------------------------------------------------------------
        if row_id.startswith("detail"):
            nested_table = row.find("table")
            if not nested_table:
                continue
            
            nested_rows = nested_table.find_all("tr")
            
            # Grab the header row (first <tr>), then the detail rows
            if len(nested_rows) > 0:
                # 1A) Header row
                header_cells = nested_rows[0].find_all(["th", "td"])
                header_texts = [hc.get_text(strip=True) for hc in header_cells]
                
                # Store the header in data_by_show if not already stored
                show_key = (current_show_name or "Unknown Show").strip()
                if show_key not in data_by_show:
                    data_by_show[show_key] = {
                        "header": ["Show", "Date"] + header_texts,  # Prepend Show & Date
                        "rows": []
                    }
                
                # 1B) Subsequent rows are actual data
                for detail_row in nested_rows[1:]:
                    data_cells = detail_row.find_all("td")
                    if data_cells:
                        row_texts = [dc.get_text(strip=True) for dc in data_cells]
                        # Build the final row with show + date + detail columns
                        final_row = [current_show_name, current_date] + row_texts
                        
                        data_by_show[show_key]["rows"].append(final_row)
        
        # ---------------------------------------------------------------------
        # 2) If this <tr> is a normal row, it should contain
        #    something like <td>Music Bank</td><td>2024-03-10</td>
        # ---------------------------------------------------------------------
        else:
            cells = row.find_all("td")
            if len(cells) >= 2:
                current_show_name = cells[2].get_text(strip=True)
                current_date = cells[1].get_text(strip=True)
                
                # If you need more columns from this row, you can parse them here as well.
    
    # -------------------------------------------------------------------------
    # After collecting all data for the year, write them out as separate CSVs
    # one CSV per show program
    # -------------------------------------------------------------------------
    for show_name, show_data in data_by_show.items():
        filename = f"{year}_{sanitize_filename(show_name)}.csv"
        print(f"Writing {filename} with {len(show_data['rows'])} rows...")
        
        with open(filename, mode="w", newline="", encoding="utf-8") as f:
            writer = csv.writer(f)
            writer.writerow(show_data["header"])  # Write the single header row
            writer.writerows(show_data["rows"])   # Write all data rows


In [15]:
test = scrape_kpopdb_year(2025)

Writing 2025_Thu.csv with 27 rows...
Writing 2025_Wed.csv with 35 rows...
Writing 2025_Tue.csv with 15 rows...
Writing 2025_Sun.csv with 41 rows...
Writing 2025_Sat.csv with 33 rows...
Writing 2025_Fri.csv with 32 rows...


In [16]:
test